In [ ]:
# !pip uninstall -y tensorflow
!pip install tensorflow-gpu==1.15.3
!pip install ../input/keras-mod2/Keras-2.3.1
!pip install keras-resnet==0.1.0
!pip install keras-retinanet==0.5.1
!nvidia-smi

In [ ]:
import numpy as np
import gzip
import pickle
import os
import glob
import time
import cv2
import datetime
import pandas as pd
from sklearn.metrics import fbeta_score
from sklearn.model_selection import KFold, train_test_split
from collections import Counter, defaultdict
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss
import random
import shutil
import operator
from PIL import Image
import platform
import json
import base64
import typing as t
import zlib
import pydicom
import re
from tqdm import tqdm

ROOT_PATH = './'
INPUT_PATH = '../input/vinbigdata-chest-xray-abnormalities-detection/'
OUTPUT_PATH = ROOT_PATH + 'modified_data_folder/'
if not os.path.isdir(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)
MODELS_PATH = ROOT_PATH + 'models_inference/'
if not os.path.isdir(MODELS_PATH):
    os.mkdir(MODELS_PATH)
CACHE_PATH = ROOT_PATH + 'cache_folder/'
if not os.path.isdir(CACHE_PATH):
    os.mkdir(CACHE_PATH)
FEATURES_PATH = ROOT_PATH + 'features_folder/'
if not os.path.isdir(FEATURES_PATH):
    os.mkdir(FEATURES_PATH)
SUBM_PATH = ROOT_PATH + 'subm_folder/'


def save_in_file(arr, file_name):
    pickle.dump(arr, gzip.open(file_name, 'wb+', compresslevel=3), protocol=4)


def load_from_file(file_name):
    return pickle.load(gzip.open(file_name, 'rb'))


def save_in_file_fast(arr, file_name):
    pickle.dump(arr, open(file_name, 'wb'), protocol=4)


def load_from_file_fast(file_name):
    return pickle.load(open(file_name, 'rb'))


def get_classes_array():
    train = pd.read_csv(INPUT_PATH + 'train.csv')
    res = dict()
    for index, row in train.iterrows():
        class_id = row['class_id']
        class_name = row['class_name']
        if class_id not in res:
            res[class_id] = class_name
        else:
            if res[class_id] != class_name:
                print('Error')
    CLASSES = []
    for i in range(15):
        CLASSES.append(res[i])
    return CLASSES


def read_single_image(path):
    try:
        img = cv2.imread(path, cv2.IMREAD_ANYDEPTH)
    except:
        print('Fail')
        return np.zeros((512, 512, 3), dtype=np.uint8)

    if len(img.shape) == 2:
        img = np.stack([img, img, img], axis=-1)

    if img.shape[2] == 2:
        img = img[:, :, :1]

    if img.shape[2] == 1:
        img = np.concatenate((img, img, img), axis=2)

    if img.shape[2] > 3:
        img = img[:, :, :3]

    return img


def get_retinanet_predictions_for_files(model_path, files, out_dir, backbone, min_width, max_width, lvl_labels):
    from keras_retinanet.utils.image import preprocess_image, resize_image
    from keras_retinanet import models

    show_debug_images = False
    show_mirror_predictions = False

    model = models.load_model(model_path, backbone_name=backbone)
    print('Proc {} files...'.format(len(files)))
    result_data = dict()
    for f in files:
        id = os.path.basename(f)[:-4]

        cache_path = out_dir + id + '.pkl'
        if os.path.isfile(cache_path):
           continue

        # try:
        image = read_single_image(f)

        if show_debug_images:
            # copy to draw on
            draw = image.copy()
            draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

        # preprocess image for network
        image = preprocess_image(image)
        image, scale = resize_image(image, min_side=min_width, max_side=max_width)

        # Add mirror
        image = np.stack((image, image[:, ::-1, :]), axis=0)

        # process image
        start = time.time()
        print('ID: {} Image shape: {} Scale: {}'.format(id, image.shape, scale))
        boxes, scores, labels = model.predict_on_batch(image)
        print('Detections shape: {} {} {}'.format(boxes.shape, scores.shape, labels.shape))
        print("Processing time: {:.2f} sec".format(time.time() - start))

        if show_debug_images:
            if show_mirror_predictions:
                draw = draw[:, ::-1, :]
            boxes_init = boxes.copy()
            boxes_init /= scale

        boxes[:, :, 0] /= image.shape[2]
        boxes[:, :, 2] /= image.shape[2]
        boxes[:, :, 1] /= image.shape[1]
        boxes[:, :, 3] /= image.shape[1]

        if show_debug_images:
            if show_mirror_predictions:
                show_image_debug(lvl_labels, draw.astype(np.uint8), boxes_init[1:], scores[1:], labels[1:])
            else:
                show_image_debug(lvl_labels, draw.astype(np.uint8), boxes_init[:1], scores[:1], labels[:1])

        # save_in_file_fast((boxes, scores, labels), cache_path)
        result_data[id] = (boxes, scores, labels)
    cache_path = out_dir + '_result.pkl'
    save_in_file_fast(result_data, cache_path)


def get_retinanet_preds_for_tst(files, model_path, backbone, reverse, min_width, max_width, out_path_prefix, labels):
    if reverse is True:
        files = files[::-1]
    get_retinanet_predictions_for_files(model_path, files, out_path_prefix + '_test', backbone, min_width, max_width, labels)


def get_retinanet_preds_for_valid(files, model_path, backbone, reverse, min_width, max_width, out_path_prefix, labels):
    if reverse is True:
        files = files[::-1]
    get_retinanet_predictions_for_files(model_path, files, out_path_prefix + '_valid', backbone, min_width, max_width, labels)


if __name__ == '__main__':

    mp = '../input/chest-xray-abnormalities-detection-models/retinanet_resnet101_sqr_removed_rads/'
    model_list = [
        mp + 'resnet101_fold_0_0.1817_05_iou_0.3_converted.h5',
        mp + 'resnet101_fold_1_0.2072_19_iou_0.3_converted.h5',
        mp + 'resnet101_fold_2_0.1938_03_iou_0.3_converted.h5',
        mp + 'resnet101_fold_3_0.1884_07_iou_0.3_converted.h5',
        mp + 'resnet101_fold_4_0.2227_05_iou_0.3_converted.h5',
    ]

    calc_fold = [3]

    for fold in range(len(model_list)):
        if fold not in calc_fold:
            continue
        print('Go fold: {}'.format(fold))
        reverse = False
        model_path = model_list[fold]
        min_width = 1024
        max_width = 1024
        backbone = 'resnet101'
        out_path_prefix = OUTPUT_PATH + os.path.basename(model_path)[:-13]
        labels = get_classes_array()
        test_files = sorted(glob.glob('../input/chest-xray-test-diff-sizes/test_png_div_2/*.png'))
        print('Files found: {}'.format(len(test_files)))
        get_retinanet_preds_for_tst(test_files, model_path, backbone, reverse, min_width, max_width, out_path_prefix, labels)

